In [1]:
import pandas as pd
import traceback

In [2]:
#EXTRACT

dfs = []
for i in range (1,11):
    for j in range(1,3):
        try:
            df = pd.read_excel(f"../data/isruc-sleep/subgroupIII/{i}/{i}_{j}.xlsx", usecols="A:I",)
            
            if not "spo2" in [str(col).lower() for col in df.columns]:
                                    print(f"../data/isruc-sleep/subgroupIII/{i}/{i}_{j}.xlsx has no header.")
                                    df = pd.read_excel(f"../data/isruc-sleep/subgroupIII/{i}/{i}_{j}.xlsx", usecols="A:I", header=None)            
            
            df["patient_id"] = i
            df["score_id"]= j
            dfs.append(df)
        except Exception as e:
            print(f"Error parsing {i}: {e}")
            traceback.print_exc()

In [3]:
# TRANSFORM
def transform(df):
    df = df.copy()
    df.columns=["epoch","stage","spo2","hr","events","bpos","txln","txex","technote", "patient_id","score_id"]

    #no point keeping points if scorer didnt score
    df.dropna(how="any", subset=['epoch','stage'], inplace=True)
    df.dropna(how="all", inplace=True)
    df.loc[df.technote.isna(), "technote"]=""
    df.loc[df.events.isna(),'events']= ""
    df['events'] = df.apply(lambda r: r['events'] + " ".join(r['hr'].split(" ")[1:]) if isinstance(r['hr'],str) else r['events'], axis=1)
    df['hr'] = df.hr.apply(lambda r: float(r.split(" ")[0]) if isinstance(r, str) else float(r))
    df.epoch = df.epoch.astype("int32")
    return df

In [4]:
dfs = [transform(df) for df in dfs]

In [5]:
df = pd.concat(dfs)

In [6]:
df.head()

epoch stage  spo2     hr events bpos txln  txex               technote  \
0      1     W     0   98.0  L Out    B    0     0  23:19:53 - LIGHTS OUT   
1      2     W     0   82.0           B    0     0                          
2      3     W     0   89.0           B    0     0                          
3      4     W     0   95.0           B    0     0                          
4      5     W     0  100.0           R    0     0                          

   patient_id  score_id  
0           1         1  
1           1         1  
2           1         1  
3           1         1  
4           1         1

In [7]:
df.to_csv("../data/isruc-sleep/subgroupIII/scoring.csv")

In [8]:
studyIII = pd.read_excel(f"../data/isruc-sleep/subgroupIII/Details_subgroup_III_Submission.xlsx", header=2,usecols="A:S")

In [9]:
studyIII.columns=[x.lower() for x in studyIII.columns]

In [10]:
studyIII['age']=pd.to_numeric(studyIII.age,errors="coerce")
studyIII['date of recording'] = pd.to_datetime(studyIII['date of recording'].str.split("-",expand=True)[0])

In [11]:
studyIII.head()

subject  age sex  diagnosis other problems  epoches     w%    n1%    n2%  \
0        1   30   M        NaN     no problem      954  17.30  12.47  39.10   
1        2   41   M        NaN     no problem      941  12.22  15.30  34.33   
2        3   52   M        NaN     no problem      824  10.80   7.89  30.34   
3        4   58   M        NaN     no problem      794  21.91  17.25  29.47   
4        5   49   F        NaN     no problem      944  31.99   7.63  31.14   

     n3%   rem%   w%.1  n1%.1  n2%.1  n3%.1  rem%.1 date of recording  
0  18.66  12.47  15.41  11.53  41.09  19.92   12.05        2009-03-28  
1  20.94  17.22   9.35  11.26  34.22  25.82   19.34        2009-04-25  
2  38.47  12.50   9.95   8.86  31.92  34.95   14.32        2009-02-14  
3  20.03  11.34  19.77  16.88  27.46  19.90   15.99        2009-03-14  
4  20.66   8.58  30.08  11.33  32.94  15.36   10.28        2009-02-14

In [12]:
#from visbrain.gui import Sleep

In [13]:
#import os

#from visbrain.gui import Sleep
#from visbrain.io import download_file, path_to_visbrain_data

In [14]:
studyIII.to_csv("../data/isruc-sleep/subgroupIII/scoring_summary.csv")